In [5]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier


In [7]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, roc_auc_score, f1_score


In [8]:
df=pd.read_csv("/Users/ravina/Desktop/CustomerChurnPrediction/Customer-Churn-Records.csv")

In [9]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [10]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname','Complain'], axis=1)

In [11]:
#df.to_csv('customerchurn_with_necessary_features.csv',index=False)

In [12]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Satisfaction Score', 'Card Type', 'Point Earned'],
      dtype='object')

In [13]:
X=df.drop(['Exited',],axis=1)
#X = Y.values
y = df['Exited']
print(X.columns)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Satisfaction Score', 'Card Type', 'Point Earned'],
      dtype='object')


In [14]:
#Split tha dataframe into categorical and numericals
def split_catg_numr(dataset):
    catg = []
    numr = []
    for i in dataset.columns:
        if len(dataset[i].unique()) > 5:
            numr.append(i)
        else:
            catg.append(i)
            
    return catg, numr
catg,numr = split_catg_numr(X)
print(catg)
print(numr)

['Geography', 'Gender', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Satisfaction Score', 'Card Type']
['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 'Point Earned']


In [15]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler()


numerical_transformer = StandardScaler()
categoricals= []

# Apply LabelEncoder to each categorical column
for column in catg:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    categoricals.append((column, le))

# Apply StandardScaler to numerical columns
X[numr] = numerical_transformer.fit_transform(X[numr])

# Concatenate numerical and encoded categorical columns
transformed_df = pd.concat([X[numr], X[catg]], axis=1)

transformed_df.head()


,CreditScore,Age,Tenure,Balance,EstimatedSalary,Point Earned,Geography,Gender,NumOfProducts,HasCrCard,IsActiveMember,Satisfaction Score,Card Type
0,-0.326221,0.293517,-1.041760,-1.225848,0.021886,-0.630839,0,0,0,1,1,1,0
1,-0.440036,0.198164,-1.387538,0.117350,0.216534,-0.666251,2,0,0,0,1,2,0
2,-1.536794,0.293517,1.032908,1.333053,0.240687,-1.015942,0,0,2,1,0,2,0
3,0.501521,0.007457,-1.387538,-1.225848,-0.108918,-1.135457,0,0,1,0,0,4,1
4,2.063884,0.388871,-1.041760,0.785728,-0.365276,-0.803472,2,0,0,1,1,4,1


In [16]:
target = df['Exited']

In [17]:
from imblearn.over_sampling import SMOTE

# Assuming X and y are defined appropriately

smote = SMOTE(sampling_strategy='minority')
X, y= smote.fit_resample(transformed_df, target)


In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [19]:
X_train.shape

(11146, 13)

# Model Building 
we will develop different classification model and will see which one give the best results. 

In [20]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [21]:
from sklearn.metrics import precision_score,recall_score, accuracy_score, roc_auc_score,f1_score,confusion_matrix

In [22]:
import warnings
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")

params = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
    'gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
}


svc = SVC()

svc_model = GridSearchCV(estimator=svc, param_grid=params, n_jobs=-1, scoring='accuracy', cv=5)

# Fit the GridSearchCV object to the data
result = svc_model.fit(X_train, y_train)

# Print the best score and parameters
print("Best: %f using %s" % (result.best_score_, result.best_params_))


Best: 0.855015 using {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}


In [4]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,roc_auc_score

svc=SVC(C=100,gamma='auto',kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_score=roc_auc_score(y_test,y_pred)
print('Recall:', recall)
print('Precision:', precision)

NameError: name 'X_train' is not defined

# AdaBoostClassifier


In [33]:
adc=AdaBoostClassifier()
adc.fit(X_train,y_train)
np.mean(cross_val_score(adc,X_train,y_train,scoring='accuracy',cv=10))


0.7710062632515292

In [34]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
params=dict()
params['n_estimators']=[50,70,100,150,200]
params['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
params['algorithm'] = ['SAMME']

adc_model=GridSearchCV(estimator=adc,param_grid=params,n_jobs=-1,scoring='accuracy',cv=5)
result=adc_model.fit(X_train,y_train)

#best score and parameters

print("Best: %f using %s" % (result.best_score_, result.best_params_))

Best: 0.768392 using {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 100}


# Observation about Adaboost performance
1. Adaboost model correctly identifies about 75.6% of the customers who actually churned.
2. Aamong all the customers predicted to churn by the model, approximately 48.9% actually churned.
3. The model achieves a moderate level of accuracy, correctly predicting around 79% of the cases across both churn and non-churn classes.

In [35]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

adc=AdaBoostClassifier(n_estimators=200,learning_rate=0.1,)
adc.fit(X_train, y_train)
y_pred = adc.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.93      0.81      0.86      2416
           1       0.49      0.76      0.59       584

    accuracy                           0.80      3000
   macro avg       0.71      0.78      0.73      3000
weighted avg       0.84      0.80      0.81      3000

Recall: 0.7551369863013698
Precision: 0.4856828193832599


# DecisionTreeClassifier 

In [36]:

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=42)

# Define parameters for GridSearchCV
params = {
    'criterion':['gini','entropy'],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV
dtc_model = GridSearchCV(estimator=dtc, param_grid=params, n_jobs=-1, scoring='accuracy', cv=5)
result = dtc_model.fit(X_train, y_train)

# Print best score and parameters
print("Best: %f using %s" % (result.best_score_, result.best_params_))


Best: 0.829157 using {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}


# Observation about the performance of Decision Tree classifier
1. Identified approximately 77.6% of the actual positive cases correctly.
2. Out of all the cases the classifier labeled as positive, approximately 48.9.06% were actually positive.
3. Accuracy indicates classifier correctly predicted 79% of the cases in the dataset.


Decision Tree and AdaboostClassifier nearly gave same results.

In [37]:
dtc = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=1,min_samples_split=2,random_state=42)
dtc.fit(X_train, y_train)
y_pred = adc.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.93      0.81      0.86      2416
           1       0.49      0.76      0.59       584

    accuracy                           0.80      3000
   macro avg       0.71      0.78      0.73      3000
weighted avg       0.84      0.80      0.81      3000

Recall: 0.7551369863013698
Precision: 0.4856828193832599


# RandomForestClassifier

In [40]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

parameters = {
              "n_estimators":[50,70,80,100,120],
              'max_depth': [3, 4, 5, 6, 7, 9, 11],
              'min_samples_split': [2, 3, 4, 5, 6, 7],
              'criterion': ['entropy', 'gini']
             }

model = RandomForestClassifier()
#gridRandomForest = RandomizedSearchCV(model, parameters, cv = 5, n_jobs = -1)
gridRandomForest = GridSearchCV(model, parameters, cv = 5, n_jobs = -1)
gridRandomForest.fit(X_train, y_train)

print("Best: %f using %s" % (gridRandomForest.best_score_, gridRandomForest.best_params_))
print('n_estimators: ',gridRandomForest.best_estimator_.n_estimators)
print('criterion: ', gridRandomForest.best_estimator_.criterion)
print('Score: ', gridRandomForest.best_score_)
print('Min Split: ', gridRandomForest.best_estimator_.min_samples_split)
print('Max depth: ', gridRandomForest.best_estimator_.max_depth)


Best: 0.900470 using {'criterion': 'gini', 'max_depth': 11, 'min_samples_split': 4, 'n_estimators': 50}
n_estimators:  50
criterion:  gini
Score:  0.9004695898322179
Min Split:  4
Max depth:  11


In [41]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42,class_weight='balanced',max_depth=11,min_samples_split=4,criterion='gini')

# Fit the model on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Accuracy: 0.8396666666666667
Confusion Matrix:
[[2142  274]
 [ 207  377]]


In [42]:
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

random_forest = RandomForestClassifier(n_estimators = 70, min_samples_split = 2, max_depth= 11,  criterion = 'gini', random_state = 0)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      2416
           1       0.59      0.64      0.61       584

    accuracy                           0.84      3000
   macro avg       0.75      0.77      0.76      3000
weighted avg       0.85      0.84      0.85      3000

Recall: 0.6386986301369864
Precision: 0.5892575039494471


In [43]:
feature_imp_random = pd.Series(random_forest.feature_importances_, index = X.columns).sort_values(ascending = False)
feature_imp_random

Age                   0.286925
NumOfProducts         0.173906
Balance               0.102750
Point Earned          0.074545
EstimatedSalary       0.072354
CreditScore           0.070420
IsActiveMember        0.052949
Geography_Germany     0.040908
Tenure                0.039045
Satisfaction Score    0.025127
Gender_Male           0.018702
Geography_Spain       0.009534
HasCrCard             0.009421
Card Type_PLATINUM    0.008079
Card Type_SILVER      0.007942
Card Type_GOLD        0.007392
dtype: float64

In [44]:
feature_importances = random_forest.feature_importances_

# Select top N features
top_features = X.columns[np.argsort(feature_importances)[::-1][:10]]
top_features

Index(['Age', 'NumOfProducts', 'Balance', 'Point Earned', 'EstimatedSalary',
       'CreditScore', 'IsActiveMember', 'Geography_Germany', 'Tenure',
       'Satisfaction Score'],
      dtype='object')

In [47]:
parameters = {'learning_rate': [0.01, 0.03, 0.05, 0.1],
              'max_depth': [1 , 2, 3, 4, 5, 6],
              'min_child_weight': [1, 3, 5, 7, 9],
              'subsample': [0.1, 0.3, 0.5, 0.7, 0.9],
              'colsample_bytree': [0.1, 0.3, 0.5, 0.7],
              'n_estimators': [500],
              'gamma': [0.1, 0.3, 0.5, 0.7],
              'reg_alpha': [0.1, 0.3, 0.5],
              'reg_lambda': [0.1, 0.3, 0.5]
             }

model = XGBClassifier()
xgb_grid =RandomizedSearchCV(model,parameters, cv = 2, n_jobs = -1)
xgb_grid.fit(X_train, y_train)

print('Score: ', xgb_grid.best_score_)
print('Params: ', xgb_grid.best_params_)

Score:  0.8405156869816084
Params:  {'subsample': 0.5, 'reg_lambda': 0.3, 'reg_alpha': 0.5, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.5}


In [48]:
xgb = XGBClassifier(subsample = 0.5, reg_lambda = 0.3, reg_alpha = 0.5, n_estimators = 500, min_child_weight = 1, max_depth = 3, learning_rate = 0.1, gamma = 0.1, colsample_bytree = 0.5, random_state = 0)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)



              precision    recall  f1-score   support

           0       0.93      0.85      0.89      2416
           1       0.54      0.72      0.61       584

    accuracy                           0.82      3000
   macro avg       0.73      0.78      0.75      3000
weighted avg       0.85      0.82      0.83      3000

Recall: 0.7157534246575342
Precision: 0.5372750642673522


# The accuracy here at XGB is also good, but same problem, the model learned a lot the negative result not the positive one. Therefore let's take random forest model as final model.

In [50]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

features = X
target = y

best_features = SelectKBest(score_func = chi2,k = 'all')
fit = best_features.fit(features,target)

featureScores = pd.DataFrame(data = fit.scores_,index = list(Y.columns),columns = ['Chi Squared Score']) 

featureScores.sort_values(by = 'Chi Squared Score', ascending = False).round(2)

,Chi Squared Score
Balance,7156628.86
EstimatedSalary,51542.37
Age,2299.99
Geography_Germany,225.01
IsActiveMember,118.54
CreditScore,102.91
Gender_Male,51.30
Geography_Spain,20.97
Point Earned,18.02
NumOfProducts,5.01


# Feature Engineering And Feature Selection
1. more insights can be driven by creating new features
2. new features can help in predicting the possible churner more accuratly



In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("/Users/ravina/Desktop/CustomerChurnPrediction/Customer-Churn-Records.csv")

In [4]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Complain',
       'Satisfaction Score', 'Card Type', 'Point Earned'],
      dtype='object')

In [5]:
X=df.drop(['Exited','Complain','RowNumber', 'CustomerId', 'Surname'],axis=1)
#X = Y.values
y = df['Exited']
print(X.columns)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Satisfaction Score', 'Card Type', 'Point Earned'],
      dtype='object')


In [6]:
X.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Satisfaction Score,Card Type,Point Earned
0,619,France,Female,42,2,0.00,1,1,1,101348.88,2,DIAMOND,464
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,3,DIAMOND,456


In [7]:
X['sufficient_balance'] = (X.EstimatedSalary - X.Balance).apply(lambda x: 0 if x >= 0 else 1)


In [8]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Satisfaction Score,Card Type,Point Earned,sufficient_balance
0,619,France,Female,42,2,0.00,1,1,1,101348.88,2,DIAMOND,464,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,3,DIAMOND,456,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,3,DIAMOND,377,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,5,GOLD,350,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,5,GOLD,425,1


In [9]:
X["is_CreditScore_low"] = X.CreditScore.apply(lambda x : 1 if x < 400 else 0)

In [10]:
X.drop(columns=['Point Earned','Satisfaction Score','Card Type'],inplace=True)

In [34]:
# we saw from EDA x['Age] is right skewwd, therefore apply log transformation.
X['Age'] = np.log1p(X.Age)

/Users/ravina/Desktop/CustomerChurnPrediction/churnvenv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [35]:
X.shape

(15924, 11)

In [36]:
X.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary',
       'Geography', 'Gender', 'NumOfProducts', 'IsActiveMember',
       'sufficient_balance', 'is_CreditScore_low'],
      dtype='object')

In [37]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15924 entries, 0 to 15923
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CreditScore         15924 non-null  float64
 1   Age                 14444 non-null  float64
 2   Tenure              15924 non-null  float64
 3   Balance             15924 non-null  float64
 4   EstimatedSalary     15924 non-null  float64
 5   Geography           15924 non-null  int64  
 6   Gender              15924 non-null  int64  
 7   NumOfProducts       15924 non-null  int64  
 8   IsActiveMember      15924 non-null  int64  
 9   sufficient_balance  15924 non-null  int64  
 10  is_CreditScore_low  15924 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 1.3 MB


In [38]:
#Split tha dataframe into categorical and numericals
def split_catg_numr(dataset):
    catg = []
    numr = []
    for i in dataset.columns:
        if len(dataset[i].unique()) > 5:
            numr.append(i)
        else:
            catg.append(i)
            
    return catg, numr
catg,numr = split_catg_numr(X)

In [39]:
print(catg)
print(numr)

['Geography', 'Gender', 'NumOfProducts', 'IsActiveMember', 'sufficient_balance', 'is_CreditScore_low']
['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']


In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder



In [46]:
X= X[catg + numr]

In [47]:


numerical_transformer = StandardScaler()
categoricals= []

# Apply LabelEncoder to each categorical column
for column in catg:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    categoricals.append((column, le))

# Apply StandardScaler to numerical columns
X[numr] = numerical_transformer.fit_transform(X[numr])

# Concatenate numerical and encoded categorical columns
transformed_df = pd.concat([X[numr], X[catg]], axis=1)

transformed_df.head()



,CreditScore,Age,Tenure,Balance,EstimatedSalary,Geography,Gender,NumOfProducts,IsActiveMember,sufficient_balance,is_CreditScore_low
0,-0.310976,0.264727,-1.037796,-1.354696,0.014691,0,0,0,1,0,0
1,-0.425984,0.178489,-1.385886,0.026532,0.211321,2,0,0,1,0,0
2,-1.534253,0.264727,1.050746,1.276656,0.235721,0,0,2,0,1,0
3,0.525453,-0.016748,-1.385886,-1.354696,-0.117446,0,0,1,0,0,0
4,2.104213,0.344828,-1.037796,0.713834,-0.376416,2,0,0,1,1,0


#Select the best categorical features using chi sqaure test

In [48]:
features = transformed_df.loc[:,catg]

In [49]:
from sklearn.feature_selection import SelectKBest, chi2

In [50]:
target = df['Exited']

best_features = SelectKBest(score_func = chi2,k = 'all')
fit = best_features.fit(features,target)

ValueError: Found input variables with inconsistent numbers of samples: [15924, 10000]

In [23]:
featureScores = pd.DataFrame(data = fit.scores_,index = list(features.columns),columns = ['Chi Squared Score']).sort_values(ascending = False,by = ["Chi Squared Score"])
featureScores

,Chi Squared Score
IsActiveMember,118.543911
is_CreditScore_low,74.228656
Gender,51.302173
sufficient_balance,27.782900
NumOfProducts,14.462867
Geography,11.701739
HasCrCard,0.143320


#Credit card doesn't have much imapact on churn, therefore let's drop this feature.

In [24]:
transformed_df.drop(columns = ["HasCrCard"],inplace = True)


In [25]:
transformed_df.head(2)

,CreditScore,Age,Tenure,Balance,EstimatedSalary,Geography,Gender,NumOfProducts,IsActiveMember,sufficient_balance,is_CreditScore_low
0,-0.326221,0.293517,-1.041760,-1.225848,0.021886,0,0,0,1,0,0
1,-0.440036,0.198164,-1.387538,0.117350,0.216534,2,0,0,1,0,0


In [26]:
cor = transformed_df.corr()
plt.figure(figsize= (12,6))
sns.heatmap(cor, annot = True, fmt=".2g",cmap = 'viridis', center = 0)
plt.show()

NameError: name 'plt' is not defined

#  Oversampling using Smote

In [27]:
from imblearn.over_sampling import SMOTE

# Assuming X and y are defined appropriately

smote = SMOTE(sampling_strategy='minority')
X, y= smote.fit_resample(transformed_df, target)


In [28]:
y.value_counts()

Exited
1    7962
0    7962
Name: count, dtype: int64

In [29]:
from sklearn.model_selection import KFold

In [23]:
fold = KFold(n_splits = 5, shuffle = True, random_state = 3)
x_train, x_test, y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 2)
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((12739, 13), (12739,), (3185, 13), (3185,))

In [33]:
# Base Estimators
mods = [XGBClassifier( random_state=0), CatBoostClassifier(verbose=False), RandomForestClassifier( random_state=0)]

# Define the number of folds for cross-validation
fold = KFold(n_splits=5, shuffle=True, random_state=42)

def get_cv_Scores(models, X, y, cv):
    scores = {}
    for model in models:
        cvscore = cross_val_score(model, X, y, cv=cv, verbose=0)
        scores[str(model)[:10]] = np.mean(cvscore)
    return pd.DataFrame(data=scores, index=["Accuracy CV"])

# Assuming X and y are defined appropriately
cv_sc = get_cv_Scores(mods, X, y, fold)
print(cv_sc)

             XGBClassif  <catboost.  RandomFore
Accuracy CV    0.900778    0.906242     0.88646


In [34]:
def get_scores(models):
    prediction_results = {}
    for model in models:
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        recall = recall_score(y_pred,y_test)
        precision = precision_score(y_pred,y_test)
        accuracy = accuracy_score(y_pred,y_test)
        roc = roc_auc_score(y_pred,y_test)
        f1 = f1_score(y_pred,y_test)
        prediction_results[str(model)[:10]] = [accuracy,precision,recall,roc,f1]
    results_df = pd.DataFrame(data = prediction_results, index = ["Accuracy", "Precision", "Recall","ROC-AUC","F1"])
    return results_df

predictions = get_scores(mods)
results_f = pd.concat([cv_sc,predictions])
results_f.style.highlight_max(axis=1,color = 'lightgreen')

,XGBClassif,<catboost.,RandomFore
Accuracy CV,0.900778,0.906242,0.886460
Accuracy,0.904553,0.905181,0.877551
Precision,0.880711,0.868655,0.874365
Recall,0.922872,0.935109,0.877707
ROC-AUC,0.905517,0.907415,0.877553
F1,0.901299,0.900658,0.876033


In [31]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
parameters = {'learning_rate': [0.01, 0.03, 0.05, 0.1],
              'max_depth': [1 , 2, 3, 4, 5, 6],
              'min_child_weight': [1, 3, 5, 7, 9],
              'subsample': [0.1, 0.3, 0.5, 0.7, 0.9],
              'colsample_bytree': [0.1, 0.3, 0.5, 0.7],
              'n_estimators': [500],
              'gamma': [0.1, 0.3, 0.5, 0.7],
              'reg_alpha': [0.1, 0.3, 0.5],
              'reg_lambda': [0.1, 0.3, 0.5]
             }

model = XGBClassifier()
xgb_grid =RandomizedSearchCV(model,parameters, cv = 2, n_jobs = -1)
xgb_grid.fit(x_train, y_train)

print('Score: ', xgb_grid.best_score_)
print('Params: ', xgb_grid.best_params_)

Score:  0.8943406088113712
Params:  {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.03, 'gamma': 0.5, 'colsample_bytree': 0.5}


In [35]:
parameters = {
              "n_estimators":[50,70,80,100,120],
              'max_depth': [3, 4, 5, 6, 7, 9, 11],
              'min_samples_split': [2, 3, 4, 5, 6, 7],
              'criterion': ['entropy', 'gini']
             }

model = RandomForestClassifier( random_state=0)
#gridRandomForest = RandomizedSearchCV(model, parameters, cv = 5, n_jobs = -1)
gridRandomForest = GridSearchCV(model, parameters, cv = 5, n_jobs = -1)
gridRandomForest.fit(x_train, y_train)

print("Best: %f using %s" % (gridRandomForest.best_score_, gridRandomForest.best_params_))
print('n_estimators: ',gridRandomForest.best_estimator_.n_estimators)
print('criterion: ', gridRandomForest.best_estimator_.criterion)
print('Score: ', gridRandomForest.best_score_)
print('Min Split: ', gridRandomForest.best_estimator_.min_samples_split)
print('Max depth: ', gridRandomForest.best_estimator_.max_depth)




Best: 0.855404 using {'criterion': 'gini', 'max_depth': 11, 'min_samples_split': 3, 'n_estimators': 100}
n_estimators:  100
criterion:  gini
Score:  0.8554040552526165
Min Split:  3
Max depth:  11


In [27]:
random_forest = RandomForestClassifier(n_estimators = 100, min_samples_split = 3, max_depth= 11,  criterion = 'gini', random_state = 0)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)

print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Recall:', recall)
print('Precision:', precision)

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      1609
           1       0.85      0.86      0.85      1576

    accuracy                           0.85      3185
   macro avg       0.85      0.85      0.85      3185
weighted avg       0.85      0.85      0.85      3185

Recall: 0.8553299492385786
Precision: 0.8472658705216844


In [36]:
X.head()

,Geography,Gender,NumOfProducts,HasCrCard,IsActiveMember,sufficient_balance,is_CreditScore_low,CreditScore,Age,Tenure,Balance,EstimatedSalary
0,France,Female,1,1,1,0,0,619,3.761200,2,0.00,101348.88
1,Spain,Female,1,0,1,0,0,608,3.737670,1,83807.86,112542.58
2,France,Female,3,1,0,1,0,502,3.761200,8,159660.80,113931.57
3,France,Female,2,0,0,0,0,699,3.688879,1,0.00,93826.63
4,Spain,Female,1,1,1,1,0,850,3.784190,2,125510.82,79084.10


In [1]:
X.drop(columns=['HasCrCard'],inplace=True)
#X.head()
final_data=pd.concat([X,df['Exited']],axis=1)
#pd.concat([X[numr], X[catg]], axis=1)

In [59]:
#final_data.head()
final_data.to_csv("feature_engineered_bankchurn_data.csv", index=False)

In [60]:
df=pd.read_csv("/Users/ravina/Desktop/CustomerChurnPrediction/research/feature_engineered_bankchurn_data.csv")
df.head(2)

,Geography,Gender,NumOfProducts,IsActiveMember,sufficient_balance,is_CreditScore_low,CreditScore,Age,Tenure,Balance,EstimatedSalary,Exited
0,France,Female,1,1,0,0,619,3.76120,2,0.00,101348.88,1
1,Spain,Female,1,1,0,0,608,3.73767,1,83807.86,112542.58,0
